In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import (LinearRegression, HuberRegressor,
                              	RANSACRegressor, TheilSenRegressor)
from sklearn.decomposition import PCA
import palettable
from pathlib import Path
import glob
import os
import scipy.ndimage as ndimage
from scipy import signal
import statsmodels.api as smapi
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### Linear Regression for Indivudual Gesture

In [7]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

#movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat']
movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat','sit_Good']
low_filt = True
g_filt = False
for move in movements:
    print(f'For {move}:')
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    #print(csv_filenames)
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    #print(tcs.iloc[:5,:])
    X = tcs.iloc[:,1:]
    y = tcs.iloc[:,0]
    
    if low_filt:
        #print('After Low pass filter:')
        #tcs = np.array(tcs)
        #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
        #tcs_filt = pd.DataFrame(tcs_filt)
        sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
        X_f = signal.sosfilt(sos, X)
    elif g_filt:
        #print('After Gaussian filter:')
        X = np.array(X)
        X_f = ndimage.gaussian_filter(X,(0,3))
    else:
        X_f = X


    #print(X_f.shape, y.shape)
    reg = LinearRegression().fit(X_f, y)
    #reg = TheilSenRegressor(random_state=42).fit(X_f, y)
    
    y_pred = reg.predict(X_f)


    mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
    rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
    mae = mean_absolute_error(y_true=y, y_pred=y_pred)
    r2 = reg.score(X_f, y)

    #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
    print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    print(f'The R2 Score of {move} for all candidates: {r2} \n')
    
    #if move == 'Leg_extension_sit':
    #regression = smapi.OLS(y, X_f).fit()
    #y_pred_ols = regression.predict(X_f)
    #mse = mean_squared_error(y_true=y, y_pred=y_pred_ols, squared=True)
    #rmse = mean_squared_error(y_true=y, y_pred=y_pred_ols, squared=False)
    #mae = mean_absolute_error(y_true=y, y_pred=y_pred_ols)
    #print('For OLS models:')
    #print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    #print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    #print(f'The R2 Score of {move} for all candidates: {r2} \n')
    #print(regression.summary())
    if move == 'Leg_extension_sit':
        for csv in csv_filenames:
            print(csv)
            t = pd.read_csv(csv,index_col = False, header = None)
            t = t.loc[:, :5]
            #print(tcs.iloc[:5,:])
            X_indi = t.iloc[:,1:]
            y_indi = t.iloc[:,0]
            #model = 
            y_pred_indi = reg.predict(X_indi)
            mse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=True)
            rmse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=False)
            mae_indi = mean_absolute_error(y_true=y_indi, y_pred=y_pred_indi)
            r2_indi = reg.score(X_indi, y_indi)
            print(f'The Root Mean Squared Error (RMSE) of {move} for {csv} candidates: {rmse_indi}')
            print(f'The Mean Absolute Error (MAE) of {move} for {csv} candidates: {mae_indi}')
            print(f'The R2 Score of {move} for {csv} candidates: {r2_indi} \n')

For HC_90:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 10.072696803362135
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.138736890168994
The R2 Score of HC_90 for all candidates: 0.8773157502694977 

For HC_45:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 7.47491448628599
The Mean Absolute Error (MAE) of HC_45 for all candidates: 4.73625740212967
The R2 Score of HC_45 for all candidates: 0.8261202895907329 

For Leg_presses_liftup:
The Root Mean Squared Error (RMSE) of Leg_presses_liftup for all candidates: 9.103975470733586
The Mean Absolute Error (MAE) of Leg_presses_liftup for all candidates: 6.302031362760883
The R2 Score of Leg_presses_liftup for all candidates: 0.9073990135866516 

For Half_squat:
The Root Mean Squared Error (RMSE) of Half_squat for all candidates: 7.8885487457643215
The Mean Absolute Error (MAE) of Half_squat for all candidates: 5.126824580842276
The R2 Score of Half_squat for all candidates: 0.867714181711885

### Linear Regression for All Gestures

In [8]:
#linear regression for all movements
all_filenames = []
for path, subdirs, files in os.walk(data_dir):
    for name in files:
        #print(os.path.join(path, name))
        all_filenames.append(os.path.join(path, name))

tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in all_filenames))
tcs = tcs.loc[:, :5]
low_filt = True
g_filt = True

X = tcs.iloc[:,1:]
y = tcs.iloc[:,0]

if low_filt:
    #tcs = np.array(tcs)
    #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
    #tcs_filt = pd.DataFrame(tcs_filt)
    sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
    X = signal.sosfilt(sos, X)
if g_filt:
    X = np.array(X)
    X = ndimage.gaussian_filter(X,(0,3))

reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)


from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
mae = mean_absolute_error(y_true=y, y_pred=y_pred)
r2 = reg.score(X, y)

#print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
print(f'The Root Mean Squared Error (RMSE) of all movements for all candidates: {rmse}')
print(f'The Mean Absolute Error (MAE) of all movements for all candidates: {mae}')
print(f'The R2 Score of all movements for all candidates: {r2} \n')

The Root Mean Squared Error (RMSE) of all movements for all candidates: 13.68171614226332
The Mean Absolute Error (MAE) of all movements for all candidates: 9.61079281064285
The R2 Score of all movements for all candidates: 0.8446589706522882 



### Try splitting the timeseries to back & forth

In [9]:
#calculate derivative of angles
dx = 0.01
de = np.gradient(y, dx)
tcs

,0,1,2,3,4,5
0,6.90174,0.00000,0.00000,0.00000,0.00000,0.00000
1,6.90543,-0.00320,0.00000,-0.00171,0.00296,-0.00156
2,6.90596,-0.00320,0.00321,0.00857,0.00148,0.00468
3,6.90250,-0.00532,-0.00427,0.00686,0.00000,-0.01088
4,6.89108,-0.00320,-0.00107,-0.00853,0.00000,-0.00311
...,...,...,...,...,...,...
26654,90.33322,0.32207,1.97199,3.44026,1.26422,0.01268
26655,90.32855,0.32924,1.96739,3.42198,1.25967,0.00321
26656,90.33280,0.33645,1.95364,3.36777,1.26422,0.00439
26657,90.33067,0.32350,1.94908,3.42198,1.27791,0.00439


In [10]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat','sit_Good']
#movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat']

low_filt = True
g_filt = False
for move in movements:
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    #print(tcs.shape)
    dx = 0.01
    de = np.gradient(tcs.iloc[:,0], dx)
    tcs['deri'] = de
    back = tcs[tcs.deri < 0].iloc[:,:5]
    forth = tcs[tcs.deri > 0].iloc[:,:5]
    for p in ['back','forth']:
        print(f'For {move} {p}:')
        if p == 'back':
            tc = back
        if p == 'forth':
            tc = forth
    
        X = tc.iloc[:,1:]
        y = tc.iloc[:,0]

        if low_filt:
            #print('After Low pass filter:')
            #tcs = np.array(tcs)
            #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
            #tcs_filt = pd.DataFrame(tcs_filt)
            sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
            X_f = signal.sosfilt(sos, X)
        if g_filt:
            #print('After Gaussian filter:')
            X = np.array(X)
            X_f = ndimage.gaussian_filter(X,(0,3))


        
        reg = LinearRegression().fit(X_f, y)
        y_pred = reg.predict(X_f)


        from sklearn.metrics import mean_squared_error
        from sklearn.metrics import mean_absolute_error
        mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
        rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        r2 = reg.score(X_f, y)

        #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
        print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
        print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
        print(f'The R2 Score of {move} for all candidates: {r2} \n')


For HC_90 back:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 11.116002009490993
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.9894027253124715
The R2 Score of HC_90 for all candidates: 0.8565020632528504 

For HC_90 forth:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 9.130583721246115
The Mean Absolute Error (MAE) of HC_90 for all candidates: 5.790788561873758
The R2 Score of HC_90 for all candidates: 0.8939778662894731 

For HC_45 back:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 8.263460854349617
The Mean Absolute Error (MAE) of HC_45 for all candidates: 5.230064021870939
The R2 Score of HC_45 for all candidates: 0.7886731315224049 

For HC_45 forth:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 6.968560181589126
The Mean Absolute Error (MAE) of HC_45 for all candidates: 4.530360543505167
The R2 Score of HC_45 for all candidates: 0.8477558329675827 

For Leg_extension_sit back:
The Root Mean Sq

# Linear Regression of individual sensors

In [14]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

#movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat']
movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat','sit_Good']
low_filt = False
g_filt = False
for move in movements:
    print(f'For {move}:')
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    #print(csv_filenames)
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    print('For Individual Sensor Linear Regression:\n')
    y = tcs.iloc[:,0]
    
    for i in range(5):
        print(f'Sensor {i+1}:')
        X = tcs.iloc[:,i+1]
        #print(X.shape)
        if low_filt:
            #print('After Low pass filter:')
            #tcs = np.array(tcs)
            #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
            #tcs_filt = pd.DataFrame(tcs_filt)
            sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
            X_f = signal.sosfilt(sos, X)
        elif g_filt:
            #print('After Gaussian filter:')
            X = np.array(X)
            X_f = ndimage.gaussian_filter(X,(0,3))
        else:
            X_f = np.array(X)
            
            X_f = X_f.reshape(-1,1)

        #print(X_f.shape, y.shape)
        reg = LinearRegression().fit(X_f, y)
        #reg = TheilSenRegressor(random_state=42).fit(X_f, y)

        y_pred = reg.predict(X_f)


        mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
        rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        r2 = reg.score(X_f, y)

        #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
        print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
        print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
        print(f'The R2 Score of {move} for all candidates: {r2} \n')


For HC_90:
For Individual Sensor Linear Regression:

Sensor 1:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 18.01721054335499
The Mean Absolute Error (MAE) of HC_90 for all candidates: 12.035276187736649
The R2 Score of HC_90 for all candidates: 0.6074704146617103 

Sensor 2:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 11.31573982878094
The Mean Absolute Error (MAE) of HC_90 for all candidates: 7.390337193738413
The R2 Score of HC_90 for all candidates: 0.8451671187567696 

Sensor 3:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 10.656476043398104
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.7918628117112085
The R2 Score of HC_90 for all candidates: 0.8626829343910498 

Sensor 4:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 13.641794222551729
The Mean Absolute Error (MAE) of HC_90 for all candidates: 9.037911699824566
The R2 Score of HC_90 for all candidates: 0.7749700743405912 

Sensor 5:
The R

# Linear Regression on Sit

In [12]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\New_Leg_extension_sit')

#movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat']
movements = ['Full_Good+Bad']
low_filt = False
g_filt = False
for move in movements:
    print(f'For {move}:')
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    #print(csv_filenames)
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    #print(tcs.iloc[:5,:])
    X = tcs.iloc[:,1:]
    y = tcs.iloc[:,0]
    
    if low_filt:
        #print('After Low pass filter:')
        #tcs = np.array(tcs)
        #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
        #tcs_filt = pd.DataFrame(tcs_filt)
        sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
        X_f = signal.sosfilt(sos, X)
    elif g_filt:
        #print('After Gaussian filter:')
        X = np.array(X)
        X_f = ndimage.gaussian_filter(X,(0,3))
    else:
        X_f = X


    #print(X_f.shape, y.shape)
    reg = LinearRegression().fit(X_f, y)
    #reg = TheilSenRegressor(random_state=42).fit(X_f, y)
    
    y_pred = reg.predict(X_f)


    mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
    rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
    mae = mean_absolute_error(y_true=y, y_pred=y_pred)
    r2 = reg.score(X_f, y)

    #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
    print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    print(f'The R2 Score of {move} for all candidates: {r2} \n')
    
    #if move == 'Leg_extension_sit':
    #regression = smapi.OLS(y, X_f).fit()
    #y_pred_ols = regression.predict(X_f)
    #mse = mean_squared_error(y_true=y, y_pred=y_pred_ols, squared=True)
    #rmse = mean_squared_error(y_true=y, y_pred=y_pred_ols, squared=False)
    #mae = mean_absolute_error(y_true=y, y_pred=y_pred_ols)
    #print('For OLS models:')
    #print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    #print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    #print(f'The R2 Score of {move} for all candidates: {r2} \n')
    #print(regression.summary())
    if move == 'Leg_extension_sit':
        for csv in csv_filenames:
            print(csv)
            t = pd.read_csv(csv,index_col = False, header = None)
            t = t.loc[:, :5]
            #print(tcs.iloc[:5,:])
            X_indi = t.iloc[:,1:]
            y_indi = t.iloc[:,0]
            #model = 
            y_pred_indi = reg.predict(X_indi)
            mse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=True)
            rmse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=False)
            mae_indi = mean_absolute_error(y_true=y_indi, y_pred=y_pred_indi)
            r2_indi = reg.score(X_indi, y_indi)
            print(f'The Root Mean Squared Error (RMSE) of {move} for {csv} candidates: {rmse_indi}')
            print(f'The Mean Absolute Error (MAE) of {move} for {csv} candidates: {mae_indi}')
            print(f'The R2 Score of {move} for {csv} candidates: {r2_indi} \n')

For Full_Good+Bad:
The Root Mean Squared Error (RMSE) of Full_Good+Bad for all candidates: 14.162538405846432
The Mean Absolute Error (MAE) of Full_Good+Bad for all candidates: 11.227769186303105
The R2 Score of Full_Good+Bad for all candidates: 0.6735110705173197 



In [17]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\New_Leg_extension_sit')

#movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat']
movements = ['Good','Full_Good+Bad']
low_filt = False
g_filt = False
for move in movements:
    print(f'For {move}:')
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    #print(csv_filenames)
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    #print(tcs.iloc[:5,:])
    X = tcs.iloc[:,1:]
    y = tcs.iloc[:,0]
    
    if low_filt:
        #print('After Low pass filter:')
        #tcs = np.array(tcs)
        #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
        #tcs_filt = pd.DataFrame(tcs_filt)
        sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
        X_f = signal.sosfilt(sos, X)
    elif g_filt:
        #print('After Gaussian filter:')
        X = np.array(X)
        X_f = ndimage.gaussian_filter(X,(0,3))
    else:
        X_f = X


    #print(X_f.shape, y.shape)
    reg = LinearRegression().fit(X_f, y)
    #reg = TheilSenRegressor(random_state=42).fit(X_f, y)
    
    y_pred = reg.predict(X_f)


    mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
    rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
    mae = mean_absolute_error(y_true=y, y_pred=y_pred)
    r2 = reg.score(X_f, y)

    #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
    print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    print(f'The R2 Score of {move} for all candidates: {r2} \n')
    
    #if move == 'Leg_extension_sit':
    #regression = smapi.OLS(y, X_f).fit()
    #y_pred_ols = regression.predict(X_f)
    #mse = mean_squared_error(y_true=y, y_pred=y_pred_ols, squared=True)
    #rmse = mean_squared_error(y_true=y, y_pred=y_pred_ols, squared=False)
    #mae = mean_absolute_error(y_true=y, y_pred=y_pred_ols)
    #print('For OLS models:')
    #print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    #print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    #print(f'The R2 Score of {move} for all candidates: {r2} \n')
    #print(regression.summary())
    if move == 'Leg_extension_sit':
        for csv in csv_filenames:
            print(csv)
            t = pd.read_csv(csv,index_col = False, header = None)
            t = t.loc[:, :5]
            #print(tcs.iloc[:5,:])
            X_indi = t.iloc[:,1:]
            y_indi = t.iloc[:,0]
            #model = 
            y_pred_indi = reg.predict(X_indi)
            mse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=True)
            rmse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=False)
            mae_indi = mean_absolute_error(y_true=y_indi, y_pred=y_pred_indi)
            r2_indi = reg.score(X_indi, y_indi)
            print(f'The Root Mean Squared Error (RMSE) of {move} for {csv} candidates: {rmse_indi}')
            print(f'The Mean Absolute Error (MAE) of {move} for {csv} candidates: {mae_indi}')
            print(f'The R2 Score of {move} for {csv} candidates: {r2_indi} \n')

For Good:
The Root Mean Squared Error (RMSE) of Good for all candidates: 10.67042167323044
The Mean Absolute Error (MAE) of Good for all candidates: 8.523709665945363
The R2 Score of Good for all candidates: 0.8056874189112258 

For Full_Good+Bad:
The Root Mean Squared Error (RMSE) of Full_Good+Bad for all candidates: 14.162538405846432
The Mean Absolute Error (MAE) of Full_Good+Bad for all candidates: 11.227769186303105
The R2 Score of Full_Good+Bad for all candidates: 0.6735110705173197 



# Sit back & forth

In [16]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\New_Leg_extension_sit')

#movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat']
movements = ['Good']

low_filt = True
g_filt = False
for move in movements:
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    #print(tcs.shape)
    dx = 0.01
    de = np.gradient(tcs.iloc[:,0], dx)
    tcs['deri'] = de
    back = tcs[tcs.deri < 0].iloc[:,:5]
    forth = tcs[tcs.deri > 0].iloc[:,:5]
    for p in ['back','forth']:
        print(f'For {move} {p}:')
        if p == 'back':
            tc = back
        if p == 'forth':
            tc = forth
    
        X = tc.iloc[:,1:]
        y = tc.iloc[:,0]

        if low_filt:
            #print('After Low pass filter:')
            #tcs = np.array(tcs)
            #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
            #tcs_filt = pd.DataFrame(tcs_filt)
            sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
            X_f = signal.sosfilt(sos, X)
        if g_filt:
            #print('After Gaussian filter:')
            X = np.array(X)
            X_f = ndimage.gaussian_filter(X,(0,3))


        
        reg = LinearRegression().fit(X_f, y)
        y_pred = reg.predict(X_f)


        from sklearn.metrics import mean_squared_error
        from sklearn.metrics import mean_absolute_error
        mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
        rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        r2 = reg.score(X_f, y)

        #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
        print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
        print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
        print(f'The R2 Score of {move} for all candidates: {r2} \n')


For Good back:
The Root Mean Squared Error (RMSE) of Good for all candidates: 15.13034487078714
The Mean Absolute Error (MAE) of Good for all candidates: 11.433760740165283
The R2 Score of Good for all candidates: 0.5917676969497491 

For Good forth:
The Root Mean Squared Error (RMSE) of Good for all candidates: 14.960612367238815
The Mean Absolute Error (MAE) of Good for all candidates: 11.92313934683194
The R2 Score of Good for all candidates: 0.6334206701146856 

